# **Exploring a Multimodal Nutritional Data Set**

This exercise provides a practical example of how a dataset from a real nutritional study could be analyzed and explored using basic statistical methods and machine learning approaches. The primary goal is to identify features that significantly influence an observed nutritional effect within the study. By applying these techniques, we aim to uncover the key factors that contribute to the outcomes, helping to enhance our understanding of nutritional impacts and guide future research. Specifically the small size of the dataset is a challenge here.

## The Dataset

The dataset we are analyzing is derived from a 2023 study titled "Distinct Microbial Taxa Are Associated with LDL-Cholesterol Reduction after 12 Weeks of Lactobacillus plantarum Intake in Mild Hypercholesterolemia: Results of a Randomized Controlled Stud" by Müller et al. This research investigated the effects of probiotic bacteria on individuals with mildly elevated LDL cholesterol levels. Key data collected includes LDL cholesterol levels before and after the 12-week study period, along with various patient features recorded at the study's onset.

Our analytical goal is to identify which key features might be linked to a significant reduction in LDL cholesterol. Data from the placebo group or from participants who left the study during execution have already been removed.

## Preprocessing
### Basic Exploration

In this section, we start by identifying the features of the multimodal nutritional dataset and determining the number of samples. Understanding the structure and volume of our dataset is essential for guiding our preprocessing and analysis efforts.
We will analyze how features in our dataset are correlated by generating a heat map. This color-coded representation will help us easily identify strong correlations, revealing potential relationships between different features.
Identifying correlated features is crucial because we plan to use a random forest model to infer feature importance. Highly correlated features can lead to inconsistent results in such models due to redundancy. To maintain the robustness of our analysis, we will remove or combine features that are highly correlated.

**Task**: Calculate the Pearson [correlation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) coefficients for each pair of features and plot them as a [heat map](https://seaborn.pydata.org/generated/seaborn.heatmap.html)!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def load_dataset(filename):
    return pd.read_csv(filename).replace("NA", pd.NA)

def plot_correlation(dataset, correlation_threshold):
    # Your code: total_correlation_matrix = ??
    high_correlation_matrix = total_correlation_matrix[total_correlation_matrix > correlation_threshold][total_correlation_matrix != 1.0]

    upper_right_mask = np.triu(np.ones_like(high_correlation_matrix, dtype=bool))

    sns.set(font_scale=1.5)
    # Your code!
    plt.rcParams['figure.figsize'] = [30, 30]
    plt.show()

dataset = load_dataset(filename='responder.csv')
plot_correlation(dataset=dataset, correlation_threshold=0.5)

### Dropping Redundant Features

As observed from the heat map, our dataset contains several highly correlated features. Some of these correlations are quite straightforward, such as total cholesterol being correlated with LDL and HDL cholesterol levels. Others, while less direct, are still plausible; for example, arterial stiffness shows a correlation with age, and dietary energy intake varies significantly with sex.

Given these observations, our next step in preprocessing is to streamline the dataset by removing redundant features. By eliminating features that are already expressed through others, we can simplify the model's complexity without losing crucial information. This reduction not only enhances the performance of our future analyses but also helps in achieving more accurate and interpretable results from the random forest model.

In [ ]:
def remove_redundant_features(dataset):
    labels = dataset.ldl_cholesterin_reduction
    redundant_features = \
    ['arterial_stiffness',                              # Already described by age
     'waist_hip_ratio', 'diet_energy',                  # Already described by sex
     'diet_mediterenian_score',                         # Already described by healthy eating score
     'blood_pressure_systolic',                         # Already described by diastolic blood pressure
     'diet_carb', 'diet_unsaturated_fat',               # Already described by diet fat
     'blood_ldl_cholesterol', 'blood_hdl_cholesterol',  # Already described by total cholesterol
     'ldl_cholesterin_reduction'                        # Remove label to prevent data leakage
    ]
    return pd.DataFrame(labels), dataset.drop(redundant_features, axis=1, errors='ignore')

dataset_labels, dataset_redundancy_removed = remove_redundant_features(dataset=dataset)
print("The columns (n={}) are:\n{}\n".format(len(dataset_redundancy_removed.columns.tolist()), dataset_redundancy_removed.columns.tolist()))
#plot_correlation(dataset=dataset_redundancy_removed, correlation_threshold=0.5)

## Data Imputation
Handling missing data is a common challenge in dataset preparation, especially when preparing data for training models. Ideally, we could exclude all samples with missing features to ensure data cleanliness, but this is not feasible due to the already limited size of our dataset. Removing too many samples could significantly impact the validity and generalizability of our model.

As an alternative, we will employ data imputation to fill in the missing values using the existing data, ensuring minimal distortion of the dataset. This method allows us to retain as much data as possible while addressing the gaps in information.

There are various techniques for data imputation, but for simplicity and effectiveness, we will use the median value of each feature to replace missing values. Using the median is robust against outliers and helps maintain the distributional characteristics of the dataset, making it a suitable choice for our purposes.

In [ ]:
def count_nan(dataset):
    # Count the number of NaN values in each column
    nan_counts = dataset.isna().sum()

    # Filter and print the counts where they are not zero
    print("Counts of 'NaN' values in each column where they exist:")
    for column, count in nan_counts.items():
        if count > 0:
            print(f"{column}: {count}")

def impute_dataset(dataset):
    # Compute the mean of each numeric column, excluding NaNs
    column_means = dataset.median()

    # Fill NaNs with the mean of their respective column
    return dataset.fillna(column_means)

count_nan(dataset=dataset_redundancy_removed)
dataset_imputed = impute_dataset(dataset_redundancy_removed)
count_nan(dataset=dataset_imputed)

### Visualizing Distribution and Range of Features
To better understand the distribution and range of values for each feature in our dataset, we will utilize boxplots. Boxplots are an excellent tool for visually summarizing the central tendency, variability, and presence of outliers within each feature.

By analyzing these boxplots, we can quickly gauge the spread of the data, identify any features with extreme values, and determine how these outliers might affect the overall analysis. This visualization step is crucial for ensuring that we fully comprehend the data's characteristics before proceeding with more complex analyses and modeling.

In [ ]:
def print_dataset_statistics(dataset):
    print("Distribution statistics for numeric columns:")
    print(dataset.agg(['min', 'max', 'mean', 'median', 'std']))

def plot_dataset_boxplots(dataset):
    # Plot a combined box plot for all standardized numeric columns
    plt.figure(figsize=(12, 8))
    dataset.plot(kind='box', vert=False)
    plt.title('Standardized Columns')
    plt.xlabel('Standardized Value')
    plt.show()
    
plot_dataset_boxplots(dataset_imputed)

### Standardization
The analysis of boxplots has revealed significant variation in the range of values across different features in our dataset. For instance, the feature representing sex ranges from 0 to 1, while age is predominantly above 60. These disparities in scale can pose challenges in modeling, as features with larger ranges might disproportionately influence the results.

To address this issue and ensure each feature contributes equally to the analysis, we will standardize the features. Standardization involves rescaling the data so that each feature has a mean of zero and a standard deviation of approximately one. This process transforms each feature into a scale where the values represent "more of that feature" or "less of that feature."

By standardizing the features, we eliminate the problem of differing scales and make the dataset more suitable for sophisticated statistical techniques and machine learning models, which often assume data is normally distributed around zero. This step is crucial for fair comparison and effective integration of features in our analyses.

**Task**: Use the sklearn [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to scale the dataset!


In [ ]:
def scale_dataset(dataset):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    # Your Code!

def preprocess_dataset(filename):
    ds = load_dataset(filename)
    return remove_redundant_features(ds)

def clean_dataset(ds):
    ds = impute_dataset(ds)
    return scale_dataset(ds)

plot_dataset_boxplots(clean_dataset(preprocess_dataset("responder.csv")[1]))


## Inferring Feature Importance by Utilizing a Random Forest
### Decision Trees

A decision tree is a machine learning model that uses a series of decisions to split data into branches and leaves. At each node of the tree, a decision is made based on a feature, directing the data down the appropriate branch until a prediction is made at a leaf node.

This model is straightforward and mimics human decision-making, making it easy to understand and interpret. However, decision trees can overfit if they grow too deep or complex, meaning they may not generalize well to new data. They are versatile for both classification and regression tasks but are often used within ensemble methods like Random Forest to enhance their accuracy and robustness.

### Random Forests

A Random Forest is an ensemble machine learning method that improves upon the simplicity of decision trees by creating a "forest" of them and aggregating their outputs. This model works by constructing multiple decision trees during training, each on a slightly different subset of the data and features. This technique, known as bootstrap aggregating or "bagging," helps in reducing the model's variance.

When making predictions, a Random Forest takes the outputs from all the individual trees and combines them to produce a more accurate and stable prediction. For classification tasks, this typically means taking a majority vote among the predictions from all trees. For regression tasks, it usually involves averaging the outputs.

The strength of a Random Forest lies in its ability to perform well on complex datasets with minimal tuning while avoiding overfitting, a common problem with individual decision trees. This robustness, combined with its straightforward interpretability (where the importance of features can be easily evaluated), makes Random Forest a popular choice for many predictive modeling tasks. (Image: "[Random forest explain](https://commons.wikimedia.org/wiki/File:Random_forest_explain.png?uselang=de)" by "TseKiChun" under [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/deed.de) license.)

![title](Random_forest_explain.png)

### Feature Importance

Feature importance in a Random Forest provides insights into which features are most influential in making predictions. This method evaluates the impact of each feature on the accuracy of the model by observing how random alterations to feature values affect the model's performance.

In a Random Forest, feature importance is typically measured by the average decrease in impurity (such as Gini impurity or entropy) across all trees in the forest when a feature is used to split the data. Alternatively, it can be assessed by examining the increase in prediction error when values of a feature are permuted, which disrupts the relationship between the feature and the target.

This approach helps in identifying features that contribute significantly to the decision-making process of the model, allowing for better understanding and optimization of the model. Feature importance is valuable for feature selection, providing a way to remove irrelevant or less important features and potentially improve model performance.

### Building a Model to Predict LDL-Cholesterol Reduction

To develop a predictive model for LDL cholesterol reduction based on our nutritional study data, we will follow several key steps:

#### Splitting the Dataset into Training and Test Sets
Before training our model, we must split the dataset into a training set and a test set. The training set is used to train the model, allowing it to learn from the data. The test set, however, serves an unbiased role in evaluating the performance of the model. This separation helps in detecting overfitting and assessing the generalizability of the model to new, unseen data.
**It's crucial to apply preprocessing steps to the training and test sets independently**. This approach prevents data leakage, where information from the test set inadvertently influences the model training process. 
#### Training the Random Forest Classifier

With our data divided and preprocessed, we will proceed to train a Random Forest classifier. This model will use the training set to learn the patterns that correlate various dietary and health-related features with reductions in LDL cholesterol.
After training, we will assess the performance of the model by calculating the loss on both the training set and the test set. This evaluation helps us understand how well the model has learned from the training data and how it performs on unseen data.

#### Extracting Feature Importance
Finally, we will extract the feature importances determined by the Random Forest. These importances reveal which features have the most influence on predicting LDL cholesterol reduction. We will visualize these importances using a plot to easily identify the most significant features according to the model. This insight is valuable for understanding the driving factors behind LDL cholesterol reduction and could inform further research and decision-making.

**Task**: There are two lines missing from the code below. Complete the code to train the [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and to compute the [validation loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)!

In [ ]:
from sklearn.model_selection import train_test_split

def split_dataset(dataset,labels,test_size):
    train, test, train_labels, test_labels = train_test_split(dataset,labels,test_size=test_size)
    test = clean_dataset(pd.DataFrame(test))
    test_labels = clean_dataset(test_labels)
    return train, test, train_labels, test_labels


labels, dataset = preprocess_dataset("responder.csv")

train_set, test_set, train_labels, test_labels = split_dataset(dataset,labels,test_size=0.001)
train_set, validation_set, train_labels, validation_labels = split_dataset(train_set,train_labels,test_size=0.25)
train_set = clean_dataset(train_set)
train_labels = clean_dataset(train_labels)

def predict_random_forest(train_set, train_labels, validation_set, validation_labels, n_estimators, max_depth):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    random_forest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    # Your code: one line to train the random forest
    training_loss = random_forest.score(train_set, train_labels)
    # Your code: one line to calculate the validation loss of the random forest as mean squared error.
    return training_loss, validation_loss, random_forest.feature_importances_

def print_feature_importance(importances, columns):
    feature_importances = pd.DataFrame(importances, index=columns, columns=['Importance'])
    sorted_feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
    sorted_feature_importances.plot(kind='bar')
    plt.title('Feature Importances')
    plt.rcParams['figure.figsize'] = [10, 10]
    plt.show()

training_loss, validation_loss, importance = predict_random_forest(train_set, train_labels, validation_set, validation_labels, n_estimators=500, max_depth=1)
print_feature_importance(importance, train_set.columns)

In Random Forest models, adjusting tree parameters like tree depth and the number of trees is essential for balancing learning depth and prediction stability. Deeper trees can learn more complex patterns, but may overfit, especially with small datasets. More trees improve prediction stability but increase computational load. With our small dataset, preventing overfitting solely through these adjustments is challenging and the feature importance through subsequence runs not very stable.

### Stabilizing Model Predictions and Feature Importance through a larger Ensemble

To improve the stability of our Random Forest model's predictions and feature importance assessments, we average results over multiple training iterations. This method reduces the influence of any specific dataset anomalies by training the model several times with varying data subsets or initial conditions. 

The averaging over the feature importance of multiple trainings can be considered an ensemble prediction of feature importance, just like the random forest is an ensemble prediction of the label! *Note*: Specifically for random forests, a similar effect could be achieved by increasing the number of trees in the forest. However, other model architectures which do not have a “forest” can be used with the following approach as well.

In [ ]:
def predict_average_random_forest(train_set, train_labels, validation_set, validation_labels, n_estimators, max_depth, n_iterations):
    # Initialize accumulators for training loss and validation loss
    total_training_loss = 0
    total_validation_loss = 0
    total_feature_importances = np.zeros(train_set.shape[1])
    for _ in range(n_iterations):
        training_loss, validation_loss, feature_importances = predict_random_forest(
            train_set, train_labels, validation_set, validation_labels, n_estimators, max_depth
        )
    
        # Accumulate the losses
        total_training_loss += training_loss
        total_validation_loss += validation_loss
    
        # Accumulate the feature importances
        total_feature_importances += feature_importances
    return total_training_loss / n_iterations, total_validation_loss / n_iterations,  total_feature_importances / n_iterations

training_loss, validation_loss, importance = predict_average_random_forest(train_set, train_labels, validation_set, validation_labels, n_estimators=50, max_depth=1, n_iterations=5)
print("Train: {}, Validiation: {}".format(training_loss, validation_loss))
print_feature_importance(importance, train_set.columns)

### Further Stabilizing Feature Importance

To further enhance the stability of feature importance assessments, we average the importance scores across multiple training and testing splits. This method minimizes the impact of variations from any single dataset configuration, ensuring a more consistent and dependable evaluation of which features most significantly influence the model's predictions. This is especially important on small datasets, as small differences in training / test split can significantly change the distribution. A more systematic approach to this is k-fold cross validation. *Note*: While this does not improve the predictions of the model, it can greatly improve our “meta knowledge” about the model and training, e.g., the average amount of overfitting or the average feature importance. (Image: "[Leave one out cross-validation animated gif for n = 8 observations.](https://commons.wikimedia.org/wiki/File:LOOCV.gif)" by "MBanuelos22" under [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/deed.en) license.)

<div>
<img src="LOOCV.gif" width="500"/>
</div>

In [ ]:
def predict_average_dataset_random_forest(train_set, train_labels, n_estimators, max_depth, tree_iterations, dataset_iterations):
    total_training_loss = 0
    total_validation_loss = 0
    total_feature_importances = np.zeros(train_set.shape[1])
    for i in range(dataset_iterations):
        local_train_set, local_validation_set, local_train_labels, local_validation_labels = split_dataset(train_set,train_labels,test_size=0.25)
        local_train_set = clean_dataset(local_train_set)
        local_train_labels = clean_dataset(local_train_labels)
        training_loss, validation_loss, importance = predict_average_random_forest(local_train_set, local_train_labels, local_validation_set, local_validation_labels, n_estimators=50, max_depth=1, n_iterations=tree_iterations)
        # Accumulate the losses
        total_training_loss += training_loss
        total_validation_loss += validation_loss
    
        # Accumulate the feature importances
        total_feature_importances += importance
    return total_training_loss / dataset_iterations, total_validation_loss / dataset_iterations,  total_feature_importances / dataset_iterations 

training_loss, validation_loss, importance = predict_average_dataset_random_forest(train_set, train_labels, n_estimators=50, max_depth=1, tree_iterations=5, dataset_iterations=5)
print("Train: {}, Validiation: {}".format(training_loss, validation_loss))
print_feature_importance(importance, train_set.columns)

### Iteratively Dropping Least Significant Features

In this experiment, we systematically drop the least significant features from our model based on their importance scores to evaluate the impact on model performance. Starting with all features, we train the model, assess feature importance, and then remove the feature with the lowest importance score. This process is repeated, each time training the model with one fewer feature. This helps to establish a rank between the features. In the case of a usable model prediction, a sudden drop in model performance can also indicate an optimal number of features.

In [ ]:
def drop_feature_experiment():
    results = []
    feature_names = train_set.columns.tolist()
    feature_drop_iterations = {feature: [] for feature in train_set.columns}
    
    while len(feature_names) >= 1:
        training_loss, validation_loss, importance = predict_average_dataset_random_forest(
            train_set[feature_names], train_labels, n_estimators=50, max_depth=2, tree_iterations=10, dataset_iterations=10
        )
    
        # Record the results
        results.append((training_loss, validation_loss, feature_names))
    
        # Drop the feature with the lowest importance
        lowest_feature = feature_names[np.argmin(importance)]
        feature_names.remove(lowest_feature)
        feature_drop_iterations[lowest_feature].append(len(feature_names))
        print("Features left: {}. Training loss: {}. Validation loss: {}, Feature eliminated: {}".format(len(feature_names), training_loss, validation_loss, lowest_feature))
    return feature_drop_iterations

print(drop_feature_experiment())

### Average Feature Rankings

By repeatedly executing the experiment of dropping the least significant features and averaging their ranks across multiple iterations, we can establish a stable ranking of feature importance. This method involves training the model multiple times, each time removing a feature based on its assessed importance from the previous iteration and recording the rank at which each feature is dropped.

To visualize the stability and variability of these rankings, we can use box plots. Each feature's rank across different iterations is represented in a box plot, showing the median rank, the interquartile range, and any outliers. This visualization helps us understand how consistently a feature is valued across multiple models and indicates the robustness of each feature's ranking.

In [ ]:
def average_drop_feature_experiment(n_runs):
    # Initialize a dictionary to store iteration counts for each feature
    feature_drop_count = {}
    
    # Run the experiment multiple times
    for i in range(n_runs):
        print(f"Experiment run {i+1}/{n_runs}")
        # Call the provided drop_feature_experiment function
        current_run_results = drop_feature_experiment()
        
        # Accumulate results from each run
        for feature, iterations in current_run_results.items():
            if feature in feature_drop_count:
                feature_drop_count[feature].extend(iterations)
            else:
                feature_drop_count[feature] = iterations
    
    return feature_drop_count

def plot_drops(average_drop_iterations):
    sorted_features = sorted(average_drop_iterations.items(), key=lambda item: np.median(item[1]))
    features_sorted, drop_iterations_sorted = zip(*sorted_features)
    features = list(average_drop_iterations.keys())
    drop_iterations = [counts for counts in average_drop_iterations.values()]
    print(features)
    print(drop_iterations)
    # Plotting the results
    plt.figure(figsize=(12, 8))
    plt.boxplot(drop_iterations_sorted, labels=features_sorted, patch_artist=True, medianprops={'linewidth': 2, 'color': 'purple'},
            boxprops={'facecolor': 'lightblue'}, whiskerprops={'linewidth': 2}, capprops={'linewidth': 2})
    plt.xlabel('Features')
    plt.ylabel('Drop Iterations')
    plt.title('Distribution of Drop Iterations for Each Feature')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

results = average_drop_feature_experiment(10)
plot_drops(results)
